In [9]:
#Right now we could maybe widdle down to movies above a certain rating + number of ratings

In [2]:
import pandas as pd
import csv
import time
import numpy as np
import itertools

In [104]:
df_ratings = pd.read_csv('data/ratings.tsv',sep = '\t')
titles = pd.read_csv('data/titles.tsv',sep='\t')
names = pd.read_csv('data/names.tsv',sep = '\t')
movies = titles[titles['titleType']=='movie']
movies = list(movies['tconst'].unique())

C:\Users\Matt\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [105]:
names.head()


,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0053137,tt0072308,tt0031983"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0037382,tt0071877,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0057345,tt0054452,tt0056404"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0072562,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050976,tt0050986,tt0060827,tt0069467"


In [91]:
##get a list of movies based on a rating threshold and year threshold
def filterMovies(ratingThreshold,yearThreshold,dfratings,dftitles):
    avgVotes = np.mean(df_ratings['numVotes'])
    qualifiedMovies = []
    len(df_ratings)
    boolean_series = dfratings.tconst.isin(movies)
    dfratings = dfratings[boolean_series]
    for row in dfratings.iterrows():
        if row[1]['averageRating'] >= ratingThreshold and row[1]['numVotes'] >= avgVotes:
            qualifiedMovies.append(row[1]['tconst'])
    for movie in qualifiedMovies:
        year = (list(dftitles[dftitles['tconst']==movie]['startYear'])[0])
        if type(year) == str:
            year = int(year)
        if year >= yearThreshold:
            qualifiedMoviesFinal.append(movie)
    return qualifiedMovies

In [92]:
qualifiedMovies = filterMovies(8.5,2000,df_ratings,titles)
print(len(qualifiedMovies))

168


In [95]:
#find the frequent actors in the list of movies that meets the support threshold
def findFreqActors(qMovies,supportThreshold):
    count = 0
    freqActors = dict()
    for chunk in pd.read_csv('data/principals.tsv' ,sep='\t', chunksize = 1000000, usecols = ['tconst','category','nconst']):
        chunk = chunk[chunk['category']=='actor']
        chunk = chunk.groupby('tconst')
        for name, group in chunk:
            if name not in qMovies:
                continue
            else:
                actors = group['nconst'].unique()
                for actor in actors:
                    if actor in freqActors:
                        freqActors[actor][0] += 1
                        freqActors[actor][1].append(name)
                    else:
                        freqActors[actor] = [1,[name]]
    for actor in freqActors:
        if freqActors[actor][0] >= supportThreshold:
            freqActorsFinal[actor] = freqActors[actor]
    return freqActors

In [96]:
freqActors = findFreqActors(qualifiedMovies,2)

In [99]:
def findPairs():
    freq_pairs = dict()
    for chunk in pd.read_csv('data/principals.tsv' ,sep='\t', chunksize = 1000000, usecols = ['tconst','category','nconst']):
        chunk = chunk[chunk['category']=='actor']
        chunk = chunk.groupby('tconst')
        for name, group in chunk:
            freqActorsGroup = [actor for actor in group['nconst'] if actor in freqActorsFinal]
            freqPairsGroup = list(itertools.combinations(freqActorsGroup,2))
            for pair in freqPairsGroup:
                if pair in freq_pairs:
                    freq_pairs[pair][0] += 1
                    freq_pairs[pair][1].append(name)
                else:
                    freq_pairs[pair] = [1,[name]]
        freq_pairs = dict(sorted(freq_pairs.items(), key=lambda item: item[1],reverse=True))
        freq_pairsFinal = dict()
        for pair in freq_pairs:
            if (pair[1],pair[0]) not in freq_pairsFinal:
                freq_pairsFinal[pair] = freq_pairs[pair]
        return freq_pairsFinal

In [100]:
freq_pairs = findPairs()

In [ ]:
for pair in freq_pairs:
    print(getName(pair[0]),getName(pair[1]))

Dragan Nikolic Aleksandar Bercek
Mark Hamill Harrison Ford
Pavle Vuisic Dragan Nikolic
Pavle Vuisic Aleksandar Bercek
Tapan Chatterjee Rabi Ghosh
Morgan Freeman Kevin Spacey
Nedumudi Venu Thilakan


In [109]:
def getName(nconst):
    return list(names[names['nconst']==nconst]['primaryName'])[0] 
def getSoloMovies(name1,name2,pairsDict,soloDict):
    name1solomovies = []
    name2solomovies = []
    for movie in soloDict[name1][1]:
        if movie not in pairsDict[(name1,name2)][1]:
            name1solomovies.append(movie)
    for movie in soloDict[name2][1]:
        if movie not in pairsDict[(name1,name2)][1]:
            name2solomovies.append(movie)
    return name1solomovies,name2solomovies
def getRatings(movies):
    ratings = []
    if len(movies) == 0:
        return 0
    for movie in movies:
        ratings.append(list(df_ratings[df_ratings['tconst']== movie]['averageRating'])[0])
    rating = np.mean(ratings)
    return rating

In [88]:
freq_pairs = dict(sorted(freq_pairs.items(), key=lambda item: item[1],reverse=True))
#('nm0482320', 'nm0007123'): 32,
count = 0
freq_pairsFinal = dict()
for pair in freq_pairs:
    if (pair[1],pair[0]) not in freq_pairsFinal:
        freq_pairsFinal[pair] = freq_pairs[pair]
print(len(freq_pairsFinal))
for pair in freq_pairsFinal:
    print(pair)
    break
print(freqActorsFinal['nm0482320'])
print(len(movies1),len(movies2))
movies1,movies2 = getSoloMovies('nm0482320','nm0893449',freq_pairsFinal,freqActorsFinal)

print(getRatings(movies1),getRatings(movies2),getRatings(freq_pairsFinal[('nm0482320', 'nm0893449')][1]))

177
('nm0482320', 'nm0893449')
[11, ['tt0119385', 'tt0133826', 'tt0214585', 'tt0214915', 'tt0237376', 'tt0249188', 'tt0255306', 'tt0282778', 'tt0291855', 'tt0292246', 'tt12361178']]
9 0
8.644444444444444 0 6.7


C:\Users\Matt\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [62]:
names[names['nconst']=='nm0007123']['primaryName']
#names[names['nconst']=='nm0007123']
#titles[titles['tconst']== 'tt0118655']

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
7096,nm0007123,Mammootty,1951,\N,"actor,producer,music_department","tt1573478,tt0097851,tt0230597,tt0271648"


In [33]:
titles['titleType'].unique()


array(['short', 'movie', 'tvShort', 'tvMovie', 'tvSeries', 'tvEpisode',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'episode'],
      dtype=object)